# Topic Modeling Song Lyrics

We will perform topic modeling using two techniques: Latent Dirichlet Allocation (LDA) and Non-negative Matrix Factorization (NMF) using tools from scikit-learn and gensim. All topic modeling code is contained in the `topic_modeling.py` script.

In [38]:
import pandas as pd
import numpy as np

In [39]:
data = pd.read_csv('data/lyrics_bob-dylan.csv')
data = data.dropna()

## Perform TFIDF Vectorization

Before we can start topic modelling, we must apply term frequency-inverse document frequency (TFIDF) vectorization to our tokenized dataset. TFIDF is used to determine how important a word is to a document in a collection or corpus ([ref](https://www.wikiwand.com/en/Tf%E2%80%93idf)). For example, let's say the word "like" is very popular across all songs. Using TFIDF, we downweight the importance of "like" because it is a word that occurs frequently within our corpus. Let's say "democracy" is another word within that song but it is very rare across all songs. Its importance would be upweighted using TFDIF because it doesn't occur very often in our corpus.

Note: scikit-learn's `TfidfVectorizer` expects an array of strings. So, we will need to concatenate our tokenized words together as a string for TFIDF to work properly. That being said, our concatenated tokenized words are very different from our original lyrics because we filtered out stopwords and performed lemmatization.

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english', min_df=3, max_df=0.9)
X = tfidf.fit_transform(data['processed_lyrics'])
print("TFIDF matrix dimensions:",X.shape)

TFIDF matrix dimensions: (590, 2463)


With scikit-learn's '`TfidfVectorizer`, you can specify a minimum and maximum document frequency (`min_df`, `max_df`). I set `min_df` to be 3, which means that a word must be mentioned in at least 3 documents in order for the vectorizer to include it. I set `max_df` to be 0.9 which will ignore words that appear in more than 90% of documents. You can think of it as a filter for corpus-specific stopwords. 

In [41]:
X

<590x2463 sparse matrix of type '<class 'numpy.float64'>'
	with 30381 stored elements in Compressed Sparse Row format>

Now that we have our TFIDF matrix, we can start topic modeling with NMF and LDA.

## Non-negative Matrix Factorization

NMF was first published in the context of machine learning of facial images by Lee and Seung in 1999. It starts with a document-word matrix, $X_{ij}$, which represents the number of occurences of word $w_i$ in document $d_j$. We create our document-word matrix $X$ using tf-idf or count vectorization. This matrix gets factorized into two smaller matrices: a word-topic matrix $W_{ik}$ and topic-document matrix $H_{kj}$. $W_{ik}$ represents the $k$ topics discovered from the documents, while $H_{kj}$ represents the coefficient weights for the topics in each document. By reducing the dimensionality of our original document-word matrix, we are able to extract information about $k$ topics. 



<img src="images/matrix_factorization.png" width="50%"/>

The process of factorizing $W$ and $H$ involves optimizing over an objective function, which in this case is the reconstruction error between $X$ and the product of its factors $W$ and $H$. $W$ and $H$ are updated iteratively until convergence (i.e., reconstruction error can no longer be minimized). In our example, a song represents one "document" in our $X$ matrix. Our goal is to reduce the dimensionality of our song-word matrix, $X$, so that we can extract meaningful $k$ topics.

In [42]:
from sklearn.decomposition import NMF

k_topics = 6
nmf = NMF(n_components=k_topics, init="nndsvd", random_state=1234)
nmf.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0,
  max_iter=200, n_components=6, random_state=1234, shuffle=False,
  solver='cd', tol=0.0001, verbose=0)

In [43]:
tfidf_features = tfidf.get_feature_names()
top_n_words = 10
word_dict = dict()

for i in range(0,k_topics):
    topic = pd.DataFrame(data={'word':tfidf_features, 'weight':nmf.components_[i]})
    sorted_topic = topic.sort_values('weight', ascending=False).head(top_n_words)
    word_dict[i+1] = list(sorted_topic['word'])

pd.DataFrame(word_dict)

,1,2,3,4,5,6
0,time,instrumental,baby,said,love,knock
1,know,lord,want,went,true,heaven
2,like,plow,lord,asked,know,knockin
3,come,home,come,took,like,door
4,tell,fixin,right,right,make,close
5,long,hand,night,clothes,heart,trying
6,heart,hold,like,dream,want,lord
7,eye,ground,mind,highway,blue,anymore
8,gone,believe,honey,little,seen,lucky
9,night,jackson,worry,shelter,need,walking


We looked at the top 10 most "relevant" words across 6 topics in our corpus of Bob Dylan. Some of the topics are hard to summarize, but others are quite obvious. For example, Topic 5 is clearly about `love`, Topic 2 seems to have some religious undertones, and Topic 6 appears to represent lyrics from Bob Dylan's song `Knockin on Heaven's Door`.

Note that results can change if you try out different $k$ topics. Choosing a small $k$ can result in extremely broad topics, while choosing a large $k$ can end up in over-clustering, which produces many highly-similar topics ([ref](https://arxiv.org/pdf/1404.4606.pdf)). There are strategies to identify optimal $k$ (e.g., term-centric stability analysis, k-clustering, etc.), but this is outside the scope of this project.

## Latent Dirichlet Allocation

Latent Dirichlet Allocation (LDA) was introduced in 2013 as a generative probabilistic model for topic discovery ([ref](http://jmlr.csail.mit.edu/papers/v3/blei03a.html)). The basic idea behind LDA is described as such:

> documents are represented as random mixtures over latent topics, where each topic is characterized
by a distribution over words

In other words, LDA treats each document as a mixture of topics, and each topic as a mixture of words. We can think of LDA as a type of `soft clustering` in which a topic represetns a "cluster" and a document's topic probabilities represent their proporition of cluster membership. Unlike k-means clustering where each document strictly belongs to a single cluster (i.e., topic), LDA allows documents to belong to a mixture of topics.

The main difference between LDA and NMF is that LDA is probabilistic in nature. Unlike NMF, LDA is a hierarchical Bayesian model with a **Dirichlet prior**. A Dirichlet distribution is a multivariate probability distribution parameterized by $\alpha$. This parameter, $\alpha$ is used to determine a document's topic mixture distribution. Assuming a symmetric Dirichlet prior, a large $\alpha$ means that each document will contain a mixture of most (or all) topics, while a small $\alpha$ means that a document will contain a mixture of a small number of topics. We can think of a very small $\alpha$ as being a type of "hard clustering" in which a document is assigned to a single topic.  

In scikit-learn's `LatentDirichletAllocation` class, $\alpha$ is described as the `doc_topic_prior` which has a default of 1/k-topics. There is also a `topic_word_prior`, also known as $\beta$, which determines a given word's topic distribution. Its default is also 1/k-topics. For the purpose of this analysis, let's keep the default priors.

In [44]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

k_topics = 6
lda = LDA(n_components=k_topics, max_iter=15, learning_method='online', random_state=1234)
lda.fit(X)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=15, mean_change_tol=0.001,
             n_components=6, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=1234, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [45]:
tfidf_features = tfidf.get_feature_names()
top_n_words = 10
word_dict = dict()
for i in range(0,k_topics):
    topic = pd.DataFrame(data={'word':tfidf_features, 'weight':lda.components_[i]})
    sorted_topic = topic.sort_values('weight', ascending=False).head(top_n_words)
    word_dict[i+1] = list(sorted_topic['word'])

pd.DataFrame(word_dict)

,1,2,3,4,5,6
0,know,ramble,success,act,knock,instrumental
1,love,santa,failure,evidently,jane,critic
2,like,claus,speaks,supposed,knockin,congressman
3,come,swing,ideal,tired,stone,rapidly
4,baby,beard,dangles,riding,heaven,prophesize
5,time,sleigh,horseman,horse,stoned,stalled
6,said,reindeer,madam,dreamin,door,writer
7,want,dawn,matchstick,whirling,breakfast,rattle
8,night,cherry,raven,myth,libido,drenched
9,tell,break,banker,swirling,subjugation,senator


LDA's results are significantly different from what we saw with NMF. The topics seem to be much more coherent and interpretable. Here are my interpretations of what each topic represents:

- Topic 1 - love
- Topic 2 - Christmas
- Topic 3 - unclear
- Topic 4 - scenes from a dream
- Topic 5 - `Knockin on Heaven's Door` (similar to NMF's Topic 6)
- Topic 6 - politics?

Some of these topics are open to interpretation and perhaps trying to optimize $$k$$ would improve topic interpretability. 

In [46]:
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda, X, tfidf, mds='tsne')
panel

PreparedData(topic_coordinates=            Freq  cluster  topics           x          y
topic                                                   
0      86.212320        1       1  -55.687599  86.619881
2       2.877156        1       2  -82.989761 -26.715849
4       2.810667        1       3  105.066818 -12.061854
5       2.799095        1       4   16.361940 -87.704376
3       2.675050        1       5   60.537960  95.676468
1       2.625713        1       6    8.657820  11.162947, topic_info=     Category      Freq          Term     Total  loglift  logprob
term                                                             
1085  Default  2.000000  instrumental  2.000000  30.0000  30.0000
996   Default  4.000000        heaven  4.000000  29.0000  29.0000
2052  Default  3.000000         stone  3.000000  28.0000  28.0000
589   Default  6.000000          door  6.000000  27.0000  27.0000
1156  Default  1.000000         knock  1.000000  26.0000  26.0000
1794  Default  1.000000         santa  1.000000  25.0000  25.0000
1040  Default  2.000000         horse  2.000000  24.0000  24.0000
1101  Default  0.000000          jane  0.000000  23.0000  23.0000
725   Default  3.000000        father  3.000000  22.0000  22.0000
507   Default  4.000000          dark  4.000000  21.0000  21.0000
214   Default  3.000000         break  3.000000  20.0000  20.0000
382   Default  4.000000          cold  4.000000  19.0000  19.0000
522   Default  4.000000          dear  4.000000  18.0000  18.0000
151   Default  4.000000         black  4.000000  17.0000  17.0000
1146  Default  4.000000          king  4.000000  16.0000  16.0000
1299  Default  4.000000          mama  4.000000  15.0000  15.0000
517   Default  3.000000           day  3.000000  14.0000  14.0000
2401  Default  3.000000        window  3.000000  13.0000  13.0000
2254  Default  5.000000       trouble  5.000000  12.0000  12.0000
516   Default  2.000000          dawn  2.000000  11.0000  11.0000
176   Default  3.000000          blow  3.000000  10.0000  10.0000
335   Default  4.000000         child  4.000000   9.0000   9.0000
926   Default  3.000000        ground  3.000000   8.0000   8.0000
1973  Default  3.000000      somebody  3.000000   7.0000   7.0000
1750  Default  3.000000          rock  3.000000   6.0000   6.0000
2450  Default  3.000000          yeah  3.000000   5.0000   5.0000
2460  Default  4.000000         young  4.000000   4.0000   4.0000
1979  Default  4.000000          soon  4.000000   3.0000   3.0000
2111  Default  3.000000          sure  3.000000   2.0000   2.0000
1342  Default  3.000000          mile  3.000000   1.0000   1.0000
...       ...       ...           ...       ...      ...      ...
1942   Topic6  0.043501        sleigh  0.568518   1.0696  -7.5620
1705   Topic6  0.041060      reindeer  0.605351   0.9490  -7.6198
484    Topic6  0.034869         cupid  0.451693   1.0784  -7.7832
331    Topic6  0.038344        cherry  0.678189   0.7670  -7.6882
516    Topic6  0.040756          dawn  2.852711  -0.6086  -7.6272
600    Topic6  0.035942         drank  0.915594   0.4022  -7.7529
963    Topic6  0.034994       happens  0.694399   0.6519  -7.7796
214    Topic6  0.038050         break  3.249222  -0.8075  -7.6959
885    Topic6  0.036954         glory  1.940721  -0.3213  -7.7251
1993   Topic6  0.035756       special  1.098089   0.2152  -7.7581
499    Topic6  0.035625        danced  1.099741   0.2100  -7.7617
299    Topic6  0.034689           cat  0.668830   0.6807  -7.7884
17     Topic6  0.034723        afford  0.707989   0.6248  -7.7874
982    Topic6  0.034634        headed  0.678411   0.6649  -7.7900
1232   Topic6  0.034719      lingered  0.740674   0.5796  -7.7875
500    Topic6  0.034103        dancer  0.523673   0.9083  -7.8054
2254   Topic6  0.037226       trouble  5.118799  -1.2839  -7.7178
11     Topic6  0.035079         adore  1.147095   0.1524  -7.7772
784    Topic6  0.034543         foggy  0.785705   0.5154  -7.7926
1423   Topic6  0.034698       newborn  0.943128   0.3373  -7.7881
172

## Comparing NMF vs. LDA

NMF and LDA are both topic modeling techniques that both extract "latent" topics from a corpus. The main difference is unlike NMF, LDA is a probabilistic model with a Dirichlet prior. In other words, NMF’s topic-word probability distribution is fixed, while LDA’s distribution varies based on how the prior is tuned (i.e., hyperparameter $\alpha$). While LDA seems to produce more coherent topics, it can have inferior performance on smaller datasets.

Unlike NMF, reconstructing $X$ with LDA is not a closed-form solution. We need to use Monte Carlo simulations to sample from the distribution of $H$ (the distribution of topics for each sample), followed by the distribution of $W$ (the distribution of words for topic $k$).

The following table is a sumamry of the key differences between NMF and LDA:

|NMF|LDA|
|---|----|
|closed-form solution|not closed form|
|deterministic|generative|
|no prior|prior hyperparameters $\alpha$ and $\beta$|